#### Applying Bat Algorithm on ANN which is trained on Iris dataset

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from datasets import load_dataset

# Load and preprocess the IMDB dataset from Hugging Face
dataset = load_dataset('imdb')

# Preprocessing and feature extraction
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
test_texts = dataset['test']['text']
test_labels = dataset['test']['label']

# Split train into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = CountVectorizer(stop_words='english', max_features=5000)
X_train = vectorizer.fit_transform(train_texts).toarray()
X_val = vectorizer.transform(val_texts).toarray()
X_test = vectorizer.transform(test_texts).toarray()

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(train_labels, dtype=torch.long)
y_val_tensor = torch.tensor(val_labels, dtype=torch.long)
y_test_tensor = torch.tensor(test_labels, dtype=torch.long)

class IMDBDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            'inputs': self.texts[idx],
            'labels': self.labels[idx]
        }

# Prepare datasets and dataloaders
train_dataset = IMDBDataset(X_train_tensor, y_train_tensor)
val_dataset = IMDBDataset(X_val_tensor, y_val_tensor)
test_dataset = IMDBDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define the ANN model
class ANNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_units, output_dim):
        super(ANNModel, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_units)
        self.fc2 = torch.nn.Linear(hidden_units, hidden_units)
        self.fc3 = torch.nn.Linear(hidden_units, output_dim)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def create_model(hidden_units, learning_rate):
    input_dim = X_train.shape[1]  # Number of features
    output_dim = 2  # Binary classification
    model = ANNModel(input_dim, int(hidden_units), output_dim)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    return model, optimizer

# Objective Function: Minimize Validation Loss
def objective_function(hyperparams):
    hidden_units = hyperparams[0]
    learning_rate = hyperparams[1]

    model, optimizer = create_model(hidden_units, learning_rate)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)

    # Train the model for one epoch
    model.train()
    for batch in train_loader:
        inputs = batch['inputs']
        labels = batch['labels']
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = torch.nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate on validation set
    model.eval()
    preds = []
    labels_list = []
    with torch.no_grad():
        for batch in val_loader:
            inputs = batch['inputs']
            labels = batch['labels']
            outputs = model(inputs)
            preds += torch.argmax(outputs, dim=1).tolist()
            labels_list += labels.tolist()

    accuracy = accuracy_score(labels_list, preds)
    return 1 - accuracy  # Minimize this value

# Bat Algorithm Implementation
class BatAlgorithm:
    def __init__(self, num_bats, max_iter, bounds, alpha=0.9, gamma=0.9):
        self.num_bats = num_bats
        self.max_iter = max_iter
        self.bounds = bounds
        self.alpha = alpha
        self.gamma = gamma

        # Initialize bats
        self.bats = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_bats, bounds.shape[0]))
        self.velocities = np.zeros((num_bats, bounds.shape[0]))
        self.fitness = np.zeros(num_bats)
        self.frequencies = np.zeros(num_bats)

        # Best bat initialization
        self.best_bat = self.bats[0]
        self.best_fitness = float("inf")

    def optimize(self, objective_function):
        for iter in range(self.max_iter):
            for i in range(self.num_bats):
                # Frequency update
                self.frequencies[i] = np.random.uniform(0, 1)
                # Velocity update
                self.velocities[i] = self.velocities[i] + (self.bats[i] - self.best_bat) * self.frequencies[i]
                # Position update
                candidate_bat = self.bats[i] + self.velocities[i]
                candidate_bat = np.clip(candidate_bat, self.bounds[:, 0], self.bounds[:, 1])

                # Evaluate candidate
                fitness = objective_function(candidate_bat)

                # Update if new solution is better
                if fitness < self.fitness[i]:
                    self.bats[i] = candidate_bat
                    self.fitness[i] = fitness

                # Update the best bat
                if self.fitness[i] < self.best_fitness:
                    self.best_bat = self.bats[i]
                    self.best_fitness = self.fitness[i]

            print(f"Iteration {iter+1}/{self.max_iter}, Best Fitness: {self.best_fitness}")

        return self.best_bat

# Hyperparameter bounds: hidden units (4 to 64), learning rate (1e-6 to 1e-2)
bounds = np.array([[4, 64], [1e-6, 1e-2]])

# Initialize and run Bat Algorithm
bat_algorithm = BatAlgorithm(num_bats=10, max_iter=10, bounds=bounds, alpha=0.9, gamma=0.9)
best_hyperparams = bat_algorithm.optimize(objective_function)
print("Best hyperparameters found:", best_hyperparams)

# Use the best hyperparameters to train the final model
best_hidden_units, best_lr = best_hyperparams
final_model, final_optimizer = create_model(best_hidden_units, best_lr)
final_train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Train and Evaluate Final Model with Best Hyperparameters
for epoch in range(3):  # You can increase the number of epochs if needed
    final_model.train()
    for batch in final_train_loader:
        inputs = batch['inputs']
        labels = batch['labels']
        final_optimizer.zero_grad()
        outputs = final_model(inputs)
        loss = torch.nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        final_optimizer.step()
    print(f"Epoch {epoch+1} completed.")

# Evaluate final model
final_model.eval()
test_loader = DataLoader(test_dataset, batch_size=32)
preds = []
labels_list = []
with torch.no_grad():
    for batch in test_loader:
        inputs = batch['inputs']
        labels = batch['labels']
        outputs = final_model(inputs)
        preds += torch.argmax(outputs, dim=1).tolist()
        labels_list += labels.tolist()

accuracy = accuracy_score(labels_list, preds)
print(f"Final model accuracy: {accuracy:.2f}")


Iteration 1/10, Best Fitness: 0.0
Iteration 2/10, Best Fitness: 0.0
Iteration 3/10, Best Fitness: 0.0
Iteration 4/10, Best Fitness: 0.0
Iteration 5/10, Best Fitness: 0.0
Iteration 6/10, Best Fitness: 0.0
Iteration 7/10, Best Fitness: 0.0
Iteration 8/10, Best Fitness: 0.0
Iteration 9/10, Best Fitness: 0.0
Iteration 10/10, Best Fitness: 0.0
Best hyperparameters found: [1.85448225e+01 5.65977599e-03]
Epoch 1 completed.
Epoch 2 completed.
Epoch 3 completed.
Final model accuracy: 0.85


#### Applying Bat Algorithm on ANN which is trained on IMDB dataset

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from scipy.optimize import minimize

# Load and preprocess the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a model function
def create_model(hidden_units, learning_rate):
    model = Sequential([
        Dense(int(hidden_units), input_dim=4, activation='relu'),
        Dense(int(hidden_units), activation='relu'),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Objective Function: Minimize Validation Loss
def objective_function(hyperparams):
    hidden_units = int(round(hyperparams[0]))  # Round and convert to integer
    learning_rate = hyperparams[1]

    model = create_model(hidden_units, learning_rate)
    model.fit(X_train, y_train, epochs=1, batch_size=32, verbose=0)  # Train for one epoch

    # Evaluate on validation set
    val_loss, val_accuracy = model.evaluate(X_test, y_test, verbose=0)
    return 1 - val_accuracy  # Minimize this value

# Bat Algorithm Implementation
class BatAlgorithm:
    def __init__(self, num_bats, max_iter, bounds, alpha=0.9, gamma=0.9):
        self.num_bats = num_bats
        self.max_iter = max_iter
        self.bounds = bounds
        self.alpha = alpha
        self.gamma = gamma

        # Initialize bats
        self.bats = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_bats, bounds.shape[0]))
        self.velocities = np.zeros((num_bats, bounds.shape[0]))
        self.fitness = np.zeros(num_bats)
        self.frequencies = np.zeros(num_bats)

        # Best bat initialization
        self.best_bat = self.bats[0]
        self.best_fitness = float("inf")

    def optimize(self, objective_function):
        for iter in range(self.max_iter):
            for i in range(self.num_bats):
                # Frequency update
                self.frequencies[i] = np.random.uniform(0, 1)
                # Velocity update
                self.velocities[i] = self.velocities[i] + (self.bats[i] - self.best_bat) * self.frequencies[i]
                # Position update
                candidate_bat = self.bats[i] + self.velocities[i]
                candidate_bat = np.clip(candidate_bat, self.bounds[:, 0], self.bounds[:, 1])

                # Evaluate candidate
                fitness = objective_function(candidate_bat)

                # Update if new solution is better
                if fitness < self.fitness[i]:
                    self.bats[i] = candidate_bat
                    self.fitness[i] = fitness

                # Update the best bat
                if self.fitness[i] < self.best_fitness:
                    self.best_bat = self.bats[i]
                    self.best_fitness = self.fitness[i]

            print(f"Iteration {iter+1}/{self.max_iter}, Best Fitness: {self.best_fitness}")

        return self.best_bat

# Hyperparameter bounds: hidden units (4 to 64), learning rate (1e-6 to 1e-2)
bounds = np.array([[4, 64], [1e-6, 1e-2]])

# Initialize and run Bat Algorithm
bat_algorithm = BatAlgorithm(num_bats=10, max_iter=10, bounds=bounds, alpha=0.9, gamma=0.9)
best_hyperparams = bat_algorithm.optimize(objective_function)
print("Best hyperparameters found:", best_hyperparams)

# Use the best hyperparameters to train the final model
best_hidden_units, best_lr = best_hyperparams
final_model = create_model(best_hidden_units, best_lr)
final_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Evaluate final model
test_loss, test_accuracy = final_model.evaluate(X_test, y_test, verbose=0)
print(f"Final model accuracy: {test_accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Iteration 1/10, Best Fitness: 0.0
Iteration 2/10, Best Fitness: 0.0
Iteration 3/10, Best Fitness: 0.0
Iteration 4/10, Best Fitness: 0.0
Iteration 5/10, Best Fitness: 0.0
Iteration 6/10, Best Fitness: 0.0
Iteration 7/10, Best Fitness: 0.0
Iteration 8/10, Best Fitness: 0.0
Iteration 9/10, Best Fitness: 0.0
Iteration 10/10, Best Fitness: 0.0
Best hyperparameters found: [1.87425958e+01 8.94280738e-03]
Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5079 - loss: 0.8729
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7962 - loss: 0.5848 
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7990 - loss: 0.4610 
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8223 - loss: 0.3952 
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8083 - loss: 0.3553 
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8846 - loss: 0.2933 
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9285 - loss: 0.2634 
Epoch 8/10
4/4 ━━━━

In [ ]:
# import numpy as np
# import torch
# from torch.utils.data import DataLoader, Dataset
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer
# from datasets import load_dataset

# # Load and preprocess the IMDB dataset
# dataset = load_dataset('imdb')

# # Preprocessing and feature extraction
# train_texts = dataset['train']['text']
# train_labels = dataset['train']['label']
# test_texts = dataset['test']['text']
# test_labels = dataset['test']['label']

# # Split train into train and validation sets
# train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

# # Vectorize the text data
# vectorizer = CountVectorizer(stop_words='english', max_features=1000)
# X_train = vectorizer.fit_transform(train_texts).toarray()
# X_val = vectorizer.transform(val_texts).toarray()
# X_test = vectorizer.transform(test_texts).toarray()

# # Convert to PyTorch tensors
# X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
# X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
# y_train_tensor = torch.tensor(train_labels, dtype=torch.long)
# y_val_tensor = torch.tensor(val_labels, dtype=torch.long)
# y_test_tensor = torch.tensor(test_labels, dtype=torch.long)

# class IMDBDataset(Dataset):
#     def __init__(self, texts, labels):
#         self.texts = texts
#         self.labels = labels

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         return {
#             'inputs': self.texts[idx],
#             'labels': self.labels[idx]
#         }

# # Prepare datasets and dataloaders
# train_dataset = IMDBDataset(X_train_tensor, y_train_tensor)
# val_dataset = IMDBDataset(X_val_tensor, y_val_tensor)
# test_dataset = IMDBDataset(X_test_tensor, y_test_tensor)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32)
# test_loader = DataLoader(test_dataset, batch_size=32)

# # Define the ANN model
# class ANNModel(torch.nn.Module):
#     def __init__(self, input_dim, hidden_units, output_dim, weights=None):
#         super(ANNModel, self).__init__()
#         self.fc1 = torch.nn.Linear(input_dim, hidden_units)
#         self.fc2 = torch.nn.Linear(hidden_units, hidden_units)
#         self.fc3 = torch.nn.Linear(hidden_units, output_dim)
#         self.relu = torch.nn.ReLU()

#         # Assign the custom weights to the first layer if provided
#         if weights is not None:
#             with torch.no_grad():
#                 self.fc1.weight.copy_(weights)

#     def forward(self, x):
#         x = self.relu(self.fc1(x))
#         x = self.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

# def create_model(hidden_units, learning_rate, weights=None):
#     input_dim = X_train.shape[1]  # Number of features
#     output_dim = 2  # Binary classification
#     model = ANNModel(input_dim, int(hidden_units), output_dim, weights)
#     optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#     return model, optimizer

# # Objective Function: Minimize Validation Loss
# def objective_function(hyperparams):
#     hidden_units = int(hyperparams[0])
#     learning_rate = hyperparams[1]

#     # Initialize weights with correct dimensions: (hidden_units, input_dim)
#     weights = torch.randn((hidden_units, X_train.shape[1]), dtype=torch.float32)

#     model, optimizer = create_model(hidden_units, learning_rate, weights)
#     criterion = torch.nn.CrossEntropyLoss()

#     # Train the model for one epoch
#     model.train()
#     for batch in train_loader:
#         inputs = batch['inputs']
#         labels = batch['labels']
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#     # Evaluate on validation set
#     model.eval()
#     preds = []
#     labels_list = []
#     with torch.no_grad():
#         for batch in val_loader:
#             inputs = batch['inputs']
#             labels = batch['labels']
#             outputs = model(inputs)
#             preds += torch.argmax(outputs, dim=1).tolist()
#             labels_list += labels.tolist()

#     accuracy = accuracy_score(labels_list, preds)
#     return 1 - accuracy  # Minimize this value

# # Bat Algorithm Implementation
# class BatAlgorithm:
#     def __init__(self, num_bats, max_iter, bounds, alpha=0.9, gamma=0.9):
#         self.num_bats = num_bats
#         self.max_iter = max_iter
#         self.bounds = bounds
#         self.alpha = alpha
#         self.gamma = gamma

#         # Initialize bats
#         self.bats = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_bats, bounds.shape[0]))
#         self.velocities = np.zeros((num_bats, bounds.shape[0]))
#         self.fitness = np.zeros(num_bats)
#         self.frequencies = np.zeros(num_bats)

#         # Best bat initialization
#         self.best_bat = self.bats[0]
#         self.best_fitness = float("inf")

#     def optimize(self, objective_function):
#         for iter in range(self.max_iter):
#             for i in range(self.num_bats):
#                 # Frequency update
#                 self.frequencies[i] = np.random.uniform(0, 1)
#                 # Velocity update
#                 self.velocities[i] = self.velocities[i] + (self.bats[i] - self.best_bat) * self.frequencies[i]
#                 # Position update
#                 candidate_bat = self.bats[i] + self.velocities[i]
#                 candidate_bat = np.clip(candidate_bat, self.bounds[:, 0], self.bounds[:, 1])

#                 # Evaluate candidate
#                 fitness = objective_function(candidate_bat)

#                 # Update if new solution is better
#                 if fitness < self.fitness[i]:
#                     self.bats[i] = candidate_bat
#                     self.fitness[i] = fitness

#                 # Update the best bat
#                 if self.fitness[i] < self.best_fitness:
#                     self.best_bat = self.bats[i]
#                     self.best_fitness = self.fitness[i]

#             print(f"Iteration {iter+1}/{self.max_iter}, Best Fitness: {self.best_fitness}")

#         return self.best_bat

# # Hyperparameter bounds: hidden units (4 to 64), learning rate (1e-6 to 1e-2)
# bounds = np.array([[4, 64], [1e-6, 1e-2]])

# # Initialize and run Bat Algorithm
# bat_algorithm = BatAlgorithm(num_bats=10, max_iter=10, bounds=bounds, alpha=0.9, gamma=0.9)
# best_hyperparams = bat_algorithm.optimize(objective_function)
# print("Best hyperparameters found:", best_hyperparams)

# # Extract the best hyperparameters
# best_hidden_units = int(best_hyperparams[0])
# best_lr = best_hyperparams[1]

# # Initialize final model with the best hyperparameters
# # Initialize weights with correct dimensions: (best_hidden_units, X_train.shape[1])
# final_weights = torch.randn((best_hidden_units, X_train.shape[1]), dtype=torch.float32)
# final_model, final_optimizer = create_model(best_hidden_units, best_lr, final_weights)

# # Train the final model
# num_epochs = 3  # You can adjust this based on your needs
# criterion = torch.nn.CrossEntropyLoss()

# for epoch in range(num_epochs):
#     final_model.train()
#     for batch in train_loader:
#         inputs = batch['inputs']
#         labels = batch['labels']
#         final_optimizer.zero_grad()
#         outputs = final_model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         final_optimizer.step()
#     print(f"Epoch {epoch+1} completed.")

# # Evaluate the final model on the test set
# final_model.eval()
# test_loader = DataLoader(test_dataset, batch_size=32)
# preds = []
# labels_list = []

# with torch.no_grad():
#     for batch in test_loader:
#         inputs = batch['inputs']
#         labels = batch['labels']
#         outputs = final_model(inputs)
#         preds += torch.argmax(outputs, dim=1).tolist()
#         labels_list += labels.tolist()

# # Calculate and print accuracy
# accuracy = accuracy_score(labels_list, preds)
# print(f"Final model accuracy: {accuracy:.2f}")


Iteration 1/10, Best Fitness: 0.0
Iteration 2/10, Best Fitness: 0.0
Iteration 3/10, Best Fitness: 0.0
Iteration 4/10, Best Fitness: 0.0
Iteration 5/10, Best Fitness: 0.0
Iteration 6/10, Best Fitness: 0.0
Iteration 7/10, Best Fitness: 0.0
Iteration 8/10, Best Fitness: 0.0
Iteration 9/10, Best Fitness: 0.0
Iteration 10/10, Best Fitness: 0.0
Best hyperparameters found: [5.16831170e+01 3.94161812e-03]
Epoch 1 completed.
Epoch 2 completed.
Epoch 3 completed.
Final model accuracy: 0.84
